# CRM: RFM Analizi

* İş Problemi: 
    * Bir e-ticaret şirketi müşterilerini segmentlere ayırıp bu segmentlere göre pazarlama stratejileri belirlemek istiyor.
* Veri Seti Hikayesi
    * https://archive.ics.uci.edu/ml/datasets/Online+Retail+II
    * Online Retail II isimli veri seti İngiltere merkezli online bir satış mağazasının 01/12/2009 - 09/12/2011 tarihleri arasındaki satışlarını içeriyor.
* Değişkenler:
    * InvoiceNo: Fatura numarası. Her işleme yani faturaya ait eşsiz numara. C ile başlıyorsa iptal edilen işlem.
    * StockCode: Ürün kodu. Her bir ürün için eşsiz numara.
    * Description: Ürün ismi
    * Quantity: Ürün adedi. Faturalardaki ürünlerden kaçar tane satıldığını ifade etmektedir.
    * InvoiceDate: Fatura tarihi ve zamanı.
    * UnitPrice: Ürün fiyatı (Sterlin cinsinden)
    * CustomerID: Eşsiz müşteri numarası
    * Country: Ülke ismi. Müşterinin yaşadığı ülke.

In [1]:
#Kullanılacak kütüphanelerin eklenmesi
import datetime as dt
import pandas as pd

In [2]:
#Gösterim ayarlarının yapılması
pd.set_option("display.max_column", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.float_format", lambda x: "%.3f" % x)

In [3]:
#Verisetinin okunması ve kopya dataframe oluşturulması
dataframe = pd.read_excel("online_retail_II.xlsx", sheet_name="Year 2009-2010")
df = dataframe.copy()
df.head(10)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.950,13085.000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.100,13085.000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.250,13085.000,United Kingdom
5,489434,22064,PINK DOUGHNUT TRINKET POT,24,2009-12-01 07:45:00,1.650,13085.000,United Kingdom
6,489434,21871,SAVE THE PLANET MUG,24,2009-12-01 07:45:00,1.250,13085.000,United Kingdom
7,489434,21523,FANCY FONT HOME SWEET HOME DOORMAT,10,2009-12-01 07:45:00,5.950,13085.000,United Kingdom
8,489435,22350,CAT BOWL,12,2009-12-01 07:46:00,2.550,13085.000,United Kingdom
9,489435,22349,"DOG BOWL , CHASING BALL DESIGN",12,2009-12-01 07:46:00,3.750,13085.000,United Kingdom


In [4]:
#Boş değerler
df.isnull().sum()
# müşteri id belli değilse müşteri segmentasyonunu bozar, bunları çıkartacağız

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

In [5]:
print("Boş değerleri çıkarmadan önce satır sayısı: ", df.shape[0])

Boş değerleri çıkarmadan önce satır sayısı:  525461


In [6]:
#Boş değerlerin çıkartılması
df.dropna(inplace=True)
print("Boş değerleri çıkarıldıktan sonra satır sayısı: ", df.shape[0])

Boş değerleri çıkarıldıktan sonra satır sayısı:  417534


In [7]:
#Dataframe'de toplam ücret ifadesi yok, eklemek için (ürünler bazında)
df["TotalPrice"] = df["Quantity"] * df["Price"]

In [8]:
#Eşsiz invoice sayısı
df["Invoice"].nunique()

23587

In [9]:
#Eşsiz müşteri sayısı
df["Customer ID"].nunique()

4383

In [10]:
#Ürünler kaçar adet satılmış
df.groupby("Description").agg({"Quantity": "sum"}).sort_values("Quantity", ascending=False).head()

,Quantity
Description,
WHITE HANGING HEART T-LIGHT HOLDER,55861
WORLD WAR 2 GLIDERS ASSTD DESIGNS,54274
BROCADE RING PURSE,47430
PACK OF 72 RETRO SPOT CAKE CASES,44507
ASSORTED COLOUR BIRD ORNAMENT,44120


In [11]:
#Fatura başına toplam kaç para
df.groupby("Invoice").agg({"TotalPrice": "sum"}).head()

,TotalPrice
Invoice,
489434,505.300
489435,145.800
489436,630.330
489437,310.750
489438,2286.240


In [12]:
#Verinin özeti
df.describe().T
#negatif değerler var, bunlar iade olduğunu gösteriyor. Bunlardan kurtulmak gerek

,count,mean,std,min,25%,50%,75%,max
Quantity,417534.000,12.759,101.220,-9360.000,2.000,4.000,12.000,19152.000
Price,417534.000,3.888,71.132,0.000,1.250,1.950,3.750,25111.090
Customer ID,417534.000,15360.645,1680.811,12346.000,13983.000,15311.000,16799.000,18287.000
TotalPrice,417534.000,19.994,99.916,-25111.090,4.250,11.250,19.350,15818.400


In [13]:
#Invoice'u C ile başlayanlar iade ürün demek
df = df[~df["Invoice"].str.contains("C", na=False)]
print("İade verileri çıkarıldıktan sonra satır sayısı: ", df.shape[0])

İade verileri çıkarıldıktan sonra satır sayısı:  407695


## RFM Metriklerinin Hesaplanması

In [14]:
#dataframedeki tarih aralıklarını öğrenmek
print("En uzak tarih:", df["InvoiceDate"].min())
print("En yakın tarih:", df["InvoiceDate"].max())

#günümüze eski bir veri seti olduğu için "bugünün tarihi" olarak daha yakın bir tercih yapmamız gerekiyor
today_date = dt.datetime(2010, 12, 11)

En uzak tarih: 2009-12-01 07:45:00
En yakın tarih: 2010-12-09 20:01:00


In [15]:
#RFM metriklerinin hesaplanması ve yeni bir dataframe'e atılması
rfm = df.groupby('Customer ID').agg({'InvoiceDate': lambda InvoiceDate: (today_date - InvoiceDate.max()).days,
                                     'Invoice': lambda Invoice: Invoice.nunique(),
                                     'TotalPrice': lambda TotalPrice: TotalPrice.sum()})
rfm.columns = ["recency", "frequency", "monetary"]
rfm.head()

,recency,frequency,monetary
Customer ID,,,
12346.000,165,11,372.860
12347.000,3,2,1323.320
12348.000,74,1,222.160
12349.000,43,3,2671.140
12351.000,11,1,300.930


In [16]:
#Daraframe özeti
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4314.000,91.270,96.944,1.000,18.000,53.000,136.000,374.000
frequency,4314.000,4.454,8.169,1.000,1.000,2.000,5.000,205.000
monetary,4314.000,2047.289,8912.523,0.000,307.950,705.550,1722.802,349164.350


In [17]:
#Monetary değerinin 0 gelmesi istediğimiz bir sonuç değil, bu kayıtları çıkartmamız lazım
rfm = rfm[rfm["monetary"] > 0]
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4312.000,91.173,96.861,1.000,18.000,53.000,136.000,374.000
frequency,4312.000,4.456,8.170,1.000,1.000,2.000,5.000,205.000
monetary,4312.000,2048.238,8914.481,2.950,307.988,706.020,1723.142,349164.350


### RFM Skorlarının Hesaplanması

In [18]:
#Recency skorunun hesaplanması
rfm["recency_score"] = pd.qcut(rfm["recency"], 5, labels=[5, 4, 3, 2, 1])

#Frequency skorunun hesaplanması
#rank kullanımı: çok tekrarlı ifadelerde (bir değerin 2 farklı kategoriye girmesi vb.) atamanın yapılması metodudur.
rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])

#Monetary skorunun hesaplanması
rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])

#RF skorunun hesaplanması
rfm["RF_SCORE"] = (rfm["recency_score"].astype(str) + rfm["frequency_score"].astype(str))

#DataFrame'in görünümü
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE
Customer ID,,,,,,,
12346.000,165,11,372.860,2,5,2,25
12347.000,3,2,1323.320,5,2,4,52
12348.000,74,1,222.160,2,1,1,21
12349.000,43,3,2671.140,3,3,5,33
12351.000,11,1,300.930,5,1,2,51


In [19]:
rfm[rfm["RF_SCORE"] == "55"].head() #şampiyonlar

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE
Customer ID,,,,,,,
12415.000,11,7,19543.840,5,5,5,55
12431.000,9,13,4370.520,5,5,5,55
12471.000,10,49,20139.740,5,5,5,55
12472.000,5,13,11308.480,5,5,5,55
12474.000,14,13,5048.660,5,5,5,55


In [20]:
#Regex ile RF_SCORE'un segment ismine dönüştürülmesi
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

rfm['segment'] = rfm['RF_SCORE'].replace(seg_map, regex=True)

In [21]:
#Segmentlere göre istatistiksel bilgiler
rfm[["segment", "recency", "frequency", "monetary"]].groupby("segment").agg(["mean", "count"])

recency       frequency       monetary      
                       mean count      mean count     mean count
segment                                                         
about_to_sleep       53.819   343     1.201   343  441.320   343
at_Risk             152.159   611     3.074   611 1188.878   611
cant_loose          124.117    77     9.117    77 4099.450    77
champions             7.119   663    12.554   663 6852.264   663
hibernating         213.886  1015     1.126  1015  403.978  1015
loyal_customers      36.287   742     6.830   742 2746.067   742
need_attention       53.266   207     2.449   207 1060.357   207
new_customers         8.580    50     1.000    50  386.199    50
potential_loyalists  18.793   517     2.017   517  729.511   517
promising            25.747    87     1.000    87  367.087    87

In [22]:
#Çıktı almak
new_df = pd.DataFrame()

new_df["new_customer_id"] = rfm[rfm["segment"] == "new_customers"].index

new_df["new_customer_id"] = new_df["new_customer_id"].astype(int)

new_df.to_csv("new_customers.csv")

rfm.to_csv("rfm.csv")

# CLTV

In [23]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

df_ = pd.read_excel("online_retail_II.xlsx", sheet_name="Year 2009-2010")
df = df_.copy()
df.head()

df.isnull().sum()
df = df[~df["Invoice"].str.contains("C", na=False)]
df.describe().T
df = df[(df['Quantity'] > 0)]
df.dropna(inplace=True)

df["TotalPrice"] = df["Quantity"] * df["Price"]

cltv_c = df.groupby('Customer ID').agg({'Invoice': lambda x: x.nunique(),
                                        'Quantity': lambda x: x.sum(),
                                        'TotalPrice': lambda x: x.sum()})

cltv_c.columns = ['total_transaction', 'total_unit', 'total_price']

cltv_c.head()

,total_transaction,total_unit,total_price
Customer ID,,,
12346.00000,11,70,372.86000
12347.00000,2,828,1323.32000
12348.00000,1,373,222.16000
12349.00000,3,993,2671.14000
12351.00000,1,261,300.93000


In [24]:
cltv_c["average_order_value"] = cltv_c["total_price"] / cltv_c["total_transaction"]
cltv_c["purchase_frequency"] = cltv_c["total_transaction"] / cltv_c.shape[0]
repeat_rate = cltv_c[cltv_c["total_transaction"] > 1].shape[0] / cltv_c.shape[0]
churn_rate = 1 - repeat_rate
cltv_c['profit_margin'] = cltv_c['total_price'] * 0.10

cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin
Customer ID,,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255,37.28600
12347.00000,2,828,1323.32000,661.66000,0.00046,132.33200
12348.00000,1,373,222.16000,222.16000,0.00023,22.21600
12349.00000,3,993,2671.14000,890.38000,0.00070,267.11400
12351.00000,1,261,300.93000,300.93000,0.00023,30.09300


In [25]:
cltv_c['customer_value'] = cltv_c['average_order_value'] * cltv_c["purchase_frequency"]
cltv_c["cltv"] = (cltv_c["customer_value"] / churn_rate) * cltv_c["profit_margin"]
cltv_c.sort_values(by="cltv", ascending=False).head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv
Customer ID,,,,,,,,
18102.00000,89,124216,349164.35000,3923.19494,0.02063,34916.43500,80.93749,8579573.77276
14646.00000,78,170342,248396.50000,3184.57051,0.01808,24839.65000,57.57916,4342070.45829
14156.00000,102,108107,196566.74000,1927.12490,0.02364,19656.67400,45.56484,2719105.08615
14911.00000,205,69722,152147.57000,742.18327,0.04752,15214.75700,35.26833,1629055.80978
13694.00000,94,125893,131443.19000,1398.33181,0.02179,13144.31900,30.46898,1215855.89003


In [26]:
cltv_c.sort_values(by="cltv", ascending=False).tail()

cltv_c["segment"] = pd.qcut(cltv_c["cltv"], 4, labels=["D", "C", "B", "A"])

cltv_c.sort_values(by="cltv", ascending=False).head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv,segment
Customer ID,,,,,,,,,
18102.00000,89,124216,349164.35000,3923.19494,0.02063,34916.43500,80.93749,8579573.77276,A
14646.00000,78,170342,248396.50000,3184.57051,0.01808,24839.65000,57.57916,4342070.45829,A
14156.00000,102,108107,196566.74000,1927.12490,0.02364,19656.67400,45.56484,2719105.08615,A
14911.00000,205,69722,152147.57000,742.18327,0.04752,15214.75700,35.26833,1629055.80978,A
13694.00000,94,125893,131443.19000,1398.33181,0.02179,13144.31900,30.46898,1215855.89003,A


In [27]:
cltv_c.groupby("segment").agg({"count", "mean", "sum"})

#cltv_c.to_csv("cltc_c.csv")

total_transaction                 total_unit                      \
                    count     mean    sum      count       mean      sum   
segment                                                                    
D                    1079  1.22892   1326       1079  109.00463   117616   
C                    1078  2.00371   2160       1078  283.05659   305135   
B                    1078  3.76902   4063       1078  680.15863   733211   
A                    1079 10.81186  11666       1079 4062.33735  4383262   

        total_price                          average_order_value            \
              count       mean           sum               count      mean   
segment                                                                      
D              1079  178.18826  192265.13000                1079 157.37300   
C              1078  475.89652  513016.45300                1078 294.45487   
B              1078 1131.35918 1219605.20000                1078 389.72832   
A              1079 6401.40546 6907116.49100                1079 670.96592   

                     purchase_frequency                 profit_margin  \
                 sum              count    mean     sum         count   
segment                                                                 
D       169805.46550               1079 0.00028 0.30737          1079   
C       317422.35514               1078 0.00046 0.50070          1078   
B       420127.12365               1078 0.00087 0.94182          1078   
A       723972.22347               1079 0.00251 2.70422          1079   

                               customer_value                     cltv  \
             mean          sum          count    mean        sum count   
segment                                                                  
D        17.81883  19226.51300           1079 0.04130   44.56772  1079   
C        47.58965  51301.64530           1078 0.11031  118.91897  1078   
B       113.13592 121960.52000           1078 0.26225  282.70867  1078   
A       640.14055 690711.64910           1079 1.48387 1601.09330  1079   

                                    
               mean            sum  
segment                             
D           2.64071     2849.32790  
C          16.86852    18184.26342  
B          96.05739   103549.86329  
A       23408.05843 25257295.04118

# CLTV Prediction

In [28]:
##########################
# Gerekli Kütüphane ve Fonksiyonlar
##########################

# !pip install lifetimes
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.4f' % x)
from sklearn.preprocessing import MinMaxScaler


def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    # dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit


#########################
# Verinin Okunması
#########################

df_ = pd.read_excel("online_retail_II.xlsx", sheet_name="Year 2010-2011")

df = df_.copy()
df.describe().T
df.head()
df.isnull().sum()

#########################
# Veri Ön İşleme
#########################

df.dropna(inplace=True)
df = df[~df["Invoice"].str.contains("C", na=False)]
df = df[df["Quantity"] > 0]
df = df[df["Price"] > 0]

replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "Price")

df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0000,2010-12-01 08:26:00,2.5500,17850.0000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6.0000,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0000,2010-12-01 08:26:00,2.7500,17850.0000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0000,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0000,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom


In [29]:
df["TotalPrice"] = df["Quantity"] * df["Price"]

today_date = dt.datetime(2011, 12, 11)

#########################
# Lifetime Veri Yapısının Hazırlanması
#########################

# recency: Son satın alma üzerinden geçen zaman. Haftalık. (kullanıcı özelinde)
# T: Müşterinin yaşı. Haftalık. (analiz tarihinden ne kadar süre önce ilk satın alma yapılmış)
# frequency: tekrar eden toplam satın alma sayısı (frequency>1)
# monetary: satın alma başına ortalama kazanç


cltv_df = df.groupby('Customer ID').agg(
    {'InvoiceDate': [lambda InvoiceDate: (InvoiceDate.max() - InvoiceDate.min()).days,
                     lambda InvoiceDate: (today_date - InvoiceDate.min()).days],
     'Invoice': lambda Invoice: Invoice.nunique(),
     'TotalPrice': lambda TotalPrice: TotalPrice.sum()})

cltv_df.columns = cltv_df.columns.droplevel(0)

cltv_df.columns = ['recency', 'T', 'frequency', 'monetary']

cltv_df["monetary"] = cltv_df["monetary"] / cltv_df["frequency"]

cltv_df.describe().T

cltv_df = cltv_df[(cltv_df['frequency'] > 1)]

cltv_df["recency"] = cltv_df["recency"] / 7

cltv_df["T"] = cltv_df["T"] / 7

In [32]:
bgf = BetaGeoFitter(penalizer_coef=0.001)

bgf.fit(cltv_df['frequency'],
        cltv_df['recency'],
        cltv_df['T'])
################################################################
# 1 hafta içinde en çok satın alma beklediğimiz 10 müşteri kimdir?
################################################################

bgf.conditional_expected_number_of_purchases_up_to_time(1,
                                                        cltv_df['frequency'],
                                                        cltv_df['recency'],
                                                        cltv_df['T']).sort_values(ascending=False).head(10)

bgf.predict(1,
            cltv_df['frequency'],
            cltv_df['recency'],
            cltv_df['T']).sort_values(ascending=False).head(10)

cltv_df["expected_purc_1_week"] = bgf.predict(1,
                                              cltv_df['frequency'],
                                              cltv_df['recency'],
                                              cltv_df['T'])

################################################################
# 1 ay içinde en çok satın alma beklediğimiz 10 müşteri kimdir?
################################################################

bgf.predict(4,
            cltv_df['frequency'],
            cltv_df['recency'],
            cltv_df['T']).sort_values(ascending=False).head(10)

cltv_df["expected_purc_1_month"] = bgf.predict(4,
                                               cltv_df['frequency'],
                                               cltv_df['recency'],
                                               cltv_df['T'])

bgf.predict(4,
            cltv_df['frequency'],
            cltv_df['recency'],
            cltv_df['T']).sum()

################################################################
# 3 Ayda Tüm Şirketin Beklenen Satış Sayısı Nedir?
################################################################

bgf.predict(4 * 3,
            cltv_df['frequency'],
            cltv_df['recency'],
            cltv_df['T']).sum()

cltv_df["expected_purc_3_month"] = bgf.predict(4 * 3,
                                               cltv_df['frequency'],
                                               cltv_df['recency'],
                                               cltv_df['T'])

cltv_df.head()

,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_purc_3_month
Customer ID,,,,,,,
12347.0000,52.1429,52.5714,7,615.7143,0.1413,0.5635,1.6784
12348.0000,40.2857,51.2857,4,442.6950,0.0920,0.3668,1.0920
12352.0000,37.1429,42.4286,8,219.5425,0.1824,0.7271,2.1631
12356.0000,43.1429,46.5714,3,937.1433,0.0862,0.3435,1.0222
12358.0000,21.2857,21.5714,2,575.2100,0.1223,0.4862,1.4388


In [33]:
ggf = GammaGammaFitter(penalizer_coef=0.01)

ggf.fit(cltv_df['frequency'], cltv_df['monetary'])

ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                        cltv_df['monetary']).head(10)

ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                        cltv_df['monetary']).sort_values(ascending=False).head(10)

cltv_df["expected_average_profit"] = ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                                                             cltv_df['monetary'])
cltv_df.sort_values("expected_average_profit", ascending=False).head(10)

,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_purc_3_month,expected_average_profit
Customer ID,,,,,,,,
12415.0000,44.7143,48.2857,21,5724.3026,0.3796,1.5139,4.5080,5772.1782
12590.0000,0.0000,30.2857,2,4591.1725,0.0115,0.0460,0.1363,5029.4196
12435.0000,26.8571,38.2857,2,3914.9450,0.0763,0.3041,0.9035,4288.9440
12409.0000,14.7143,26.1429,3,3690.8900,0.1174,0.4674,1.3854,3918.8128
14088.0000,44.5714,46.1429,13,3864.5546,0.2603,1.0379,3.0896,3917.1297
18102.0000,52.2857,52.5714,60,3859.7391,0.9685,3.8636,11.5112,3870.9969
12753.0000,48.4286,51.8571,6,3571.5650,0.1261,0.5028,1.4973,3678.5783
14646.0000,50.4286,50.7143,73,3646.0757,1.2064,4.8119,14.3340,3654.8148
15749.0000,13.8571,47.5714,3,3028.7800,0.0280,0.1116,0.3320,3216.0523


In [34]:
cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency'],
                                   cltv_df['T'],
                                   cltv_df['monetary'],
                                   time=3,  # 3 aylık
                                   freq="W",  # T'nin frekans bilgisi.
                                   discount_rate=0.01)

cltv.head()

cltv = cltv.reset_index()

cltv_final = cltv_df.merge(cltv, on="Customer ID", how="left")
cltv_final.sort_values(by="clv", ascending=False).head(10)

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_purc_3_month,expected_average_profit,clv
1122,14646.0000,50.4286,50.7143,73,3646.0757,1.2064,4.8119,14.3340,3654.8148,55741.0845
2761,18102.0000,52.2857,52.5714,60,3859.7391,0.9685,3.8636,11.5112,3870.9969,47412.5801
843,14096.0000,13.8571,14.5714,17,3163.5882,0.7287,2.8955,8.5526,3196.4361,29061.6614
36,12415.0000,44.7143,48.2857,21,5724.3026,0.3796,1.5139,4.5080,5772.1782,27685.1000
1257,14911.0000,53.1429,53.4286,201,691.7101,3.1264,12.4722,37.1641,692.3264,27377.4115
2458,17450.0000,51.2857,52.5714,46,2863.2749,0.7474,2.9815,8.8830,2874.1987,27166.0643
874,14156.0000,51.5714,53.1429,55,2104.0267,0.8775,3.5005,10.4298,2110.7542,23424.4032
2487,17511.0000,52.8571,53.4286,31,2933.9431,0.5088,2.0298,6.0476,2950.5801,18986.6123
2075,16684.0000,50.4286,51.2857,28,2209.9691,0.4781,1.9068,5.6801,2223.8850,13440.4131
650,13694.0000,52.7143,53.4286,50,1275.7005,0.8008,3.1946,9.5186,1280.2183,12966.1347


In [35]:
##############################################################
# 5. CLTV'ye Göre Segmentlerin Oluşturulması
##############################################################

cltv_final

cltv_final["segment"] = pd.qcut(cltv_final["clv"], 4, labels=["D", "C", "B", "A"])

cltv_final.sort_values(by="clv", ascending=False).head(50)

cltv_final.groupby("segment").agg(
    {"count", "mean", "sum"})

Customer ID                          recency                        T                    frequency               monetary                      expected_purc_1_week                 expected_purc_1_month                 expected_purc_3_month                  expected_average_profit                        clv                       
              count       mean           sum   count    mean        sum count    mean        sum     count    mean   sum    count     mean         sum                count   mean      sum                 count   mean      sum                 count   mean       sum                   count     mean         sum count      mean          sum
segment                                                                                                                                                                                                                                                                                                                                           
D               712 15558.4761 11077635.0000     712 22.0738 15716.5714   712 40.4649 28811.0000       712  3.0646  2182      712 183.9631 130981.7123                  712 0.0711  50.5898                   712 0.2830 201.4774                   712 0.8400  598.0905                     712 199.4377 141999.6650   712  143.2970  102027.4901
C               711 15309.6343 10885150.0000     711 30.6697 21806.1429   711 38.1111 27097.0000       711  4.0956  2912      711 271.6948 193174.9847                  711 0.1206  85.7309                   711 0.4802 341.3936                   711 1.4251 1013.2176                     711 289.9856 206179.7461   711  380.7919  270743.0065
B               711 15352.8186 10915854.0000     711 29.5148 20985.0000   711 34.8117 24751.1429       711  5.4416  3869      711 373.4425 265517.6312                  711 0.1625 115.5066                   711 0.6465 459.6926                   711 1.9165 1362.6595                     711 393.8944 280058.9431   711  688.2650  489356.3925
A               711 14947.3586 10627572.0000     711 31.4109 22333.1429   711 34.4840 24518.1429       711 11.3586  8076      711 659.8586 469159.4581                  711 0.2736 194.4993                   711 1.0891 774.3298                   711 3.2309 2297.1448                     711 685.8998 487674.7239   711 2222.3600 1580097.9284